<a href="https://colab.research.google.com/github/Cesare-Caputo/Tote/blob/main/BERT_model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -q is for silent output
!pip install horovod

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for horovod
  Running setup.py clean for horovod
Failed to build horovod
ERROR: Could not build wheels for horovod, which is required to install pyproject.toml-based projects


In [ ]:
import horovod.tensorflow.keras as hvd

ModuleNotFoundError: ignored

# Import Packages

Import the neccessary packages for the project


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense
from transformers import BertTokenizer
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from transformers import TFBertModel

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
lines = pd.read_csv(r"C:\BC personal\coding\Bert example tags expanded.csv", header=0)
lines = lines.sample(frac=1)
lines
txt = lines["Text"].tolist()
text = []
for line in txt:
    text.append(line.lower())
text
tags = []
slot_map = {}
slot_names = ["[PAD]"]
slot_map["[PAD]"] = 0
for i in range(len(lines)):
    k = ""
    for j in range(11):
#        print(i,j)
        slot = lines.iloc[i,j+1]
#        print(slot)
        if pd.isna(slot):
            break
        else:
            if slot not in slot_names:
                slot_names.append(slot)
                slot_map[slot] = len(slot_map)
            k = k + " " + str(slot)
    tags.append(k)
print(slot_names)
print(slot_map)
def encode_token_labels(text_sequences, slot_names, tokenizer, slot_map,
                        max_length):
    encoded = np.zeros(shape=(len(text_sequences), max_length), dtype=np.int32)
    for i, (text_sequence, word_labels) in enumerate(
            zip(text_sequences, slot_names)):
        encoded_labels = []
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            #print(word, word_label)
            tokens = tokenizer.tokenize(word)
            #print(tokens)
            encoded_labels.append(slot_map[word_label])
            #print(encoded_labels)
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
        print(tokenizer.tokenize(text_sequence))
        print(encoded_labels)
    return encoded
#slot_train = encode_token_labels(text, tags, tokenizer, slot_map, 25)
slot_train = encode_token_labels(text[:75], tags, tokenizer, slot_map, 25)
slot_test = encode_token_labels(text[75:], tags, tokenizer, slot_map, 25)
def encode_dataset(tokenizer, text_sequences, max_length):
    token_ids = np.zeros(shape=(len(text_sequences), max_length),
                         dtype=np.int32)
    for i, text_sequence in enumerate(text_sequences):
        print(text_sequence)
        encoded = tokenizer.encode(text_sequence)
        #print(len(encoded))
        token_ids[i, 0:len(encoded)] = encoded
    attention_masks = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_masks": attention_masks}
encoded_train = encode_dataset(tokenizer, text[:75], 25)
encoded_test = encode_dataset(tokenizer, text[75:], 25)
class JointIntentAndSlotFillingModel(tf.keras.Model):
    def __init__(self, slot_num_labels=None, model_name="bert-base-uncased", dropout_prob=0.1):
        print("started init")
        super().__init__(name="joint_intent_slot")
        print(model_name)
        self.bert = TFBertModel.from_pretrained(model_name)
        print("bert")
        self.dropout = Dropout(dropout_prob)
        print("dropout")
        #self.intent_classifier = Dense(intent_num_labels,
         #                              name="intent_classifier")
        self.slot_classifier = Dense(slot_num_labels, name="slot_classifier")
        #print("init over")

    #@tf.function(input_signature=[None])
    def call(self, inputs, **kwargs):
        print("started call")
        sequence_output, pooled_output = self.bert(inputs, **kwargs)
        #sequence_output = self.bert(inputs, **kwargs)
        # The first output of the main
        # and gives a "pooled" representation for the full sequence from the
        # hidden state that corresponds tBERT layer has shape:
        # (batch_size, max_length, output_dim)
        sequence_output = self.dropout(sequence_output, training=kwargs.get("training", False))
        slot_logits = self.slot_classifier(sequence_output)

        # The second output of the main BERT layer has shape:
        # (batch_size, output_dim)o the "[CLS]" token.
        pooled_output = self.dropout(pooled_output, training=kwargs.get("training", False))
        ##intent_logits = self.intent_classifier(pooled_output)

        #return slot_logits, intent_logits
        return slot_logits
print(len(slot_map))
joint_model = JointIntentAndSlotFillingModel(slot_num_labels=len(slot_map))
print("Done")
opt = Adam(learning_rate=3e-5, epsilon=1e-08)
print("Done")
losses = [SparseCategoricalCrossentropy(from_logits=True)]#, SparseCategoricalCrossentropy(from_logits=True)]
print("Done")
metrics = [SparseCategoricalAccuracy('accuracy')]
print("Done")
joint_model.compile(optimizer=opt, loss=losses, metrics=metrics)
print("Done")
history = joint_model.fit(encoded_train, slot_train, epochs=15, batch_size=32)
history = joint_model.evaluate(encoded_test, slot_test)
print(history)


FileNotFoundError: ignored